In [7]:
import csv
import pandas as pd

data = pd.read_csv('data/GlobalLandTemperaturesByCity.csv',sep=',',index_col=False)

poland_data = data[data['Country'] == 'Poland']

poland_data.to_csv('poland_data.csv',index=False)

In [9]:
# Listing unique cities in poland_data

list_city = list(pd.unique(poland_data['City']))
list_city

['Bialystok',
 'Bielsko Biala',
 'Bydgoszcz',
 'Bytom',
 'Chorzow',
 'Cracow',
 'Czestochowa',
 'Dabrowa Gornicza',
 'Elblag',
 'Gdansk',
 'Gdynia',
 'Gliwice',
 'Gorzow Wielkopolski',
 'Kalisz',
 'Katowice',
 'Kielce',
 'Koszalin',
 'Legnica',
 'Lublin',
 'Olsztyn',
 'Opole',
 'Plock',
 'Poznan',
 'Radom',
 'Ruda Slaska',
 'Rybnik',
 'Rzeszow',
 'Sosnowiec',
 'Szczecin',
 'Tarnow',
 'Torun',
 'Tychy',
 'Walbrzych',
 'Warsaw',
 'Wloclawek',
 'Wroclaw',
 'Zabrze',
 'Zielona Gora']